## Here's a Recap on Monitoring and Self-Healing with Probes

- On each node in a kubernetes cluster, the **kubelet** monitors the **health of Pods** and re-starts them when necessary.

<img src=../notebook_images/pod_crash_restart.png width="600" height="200" style="margin: 1em" />

- But the **kubelet** can't determine the **health of a container running inside a Pod** without additional help.

<img src=../notebook_images/container_crash_no_restart.png width="600" height="200" style="margin: 1em" />

- In the **containers** section of a Pod (or Pod template) definition, each container can define three different types of **Probes**.
  - <span style="color:#DDDD00;font-weight:bold">startupProbe</span> : used to determine if the container's microservice is considered initialzed and **started**.
    - A **failed startup probe** will **restart the container**.
  - <span style="color:#00C800;font-weight:bold">readinessProbe</span> : used to determine if the container's microservice is considered  **ready to accept traffic**.
    - A **failed readiness probe** will **stop the container from receiving traffic** (it will not restart the container).
  - <span style="color:#6688FF;font-weight:bold">livenessProbe</span> : used to determine if the container's microservice is considered **alive**.
    - A **failed liveness probe** will **restart the container**.

<img src=../notebook_images/pod_with_probes.png width="900" height="500" style="margin: 1em" />

- The **kubelet** will **use the probes to determine a container's health** (or more importantly, the **health of the microservice** it contains).
  - The **kubelet** checks a container periodically using the configured **probes**.
  - The **kubelet** will **restart a Pod** who's container's **liveness proble** is considered **failed**.

<img src=../notebook_images/kubelet_container_probing.png width="300" height="150" style="margin: 1em" />

- Each probe can be configured to use one of three **probe actions**.
  - <span style="color:#DDDD00;font-weight:bold">exec</span> action will execute a command inside the container.
    - The action **succeeds** if the commands **exit code is 0**.
    - The action **fails** if the commands **exit code is not 0**.
  - <span style="color:#00C800;font-weight:bold">tcpSocket</span> action will Check if a TCP port is open in the container.
    - The action **succeeds** if the TCP **port is open**.
    - The action **fails** if the TCP **port is not open**.
  - <span style="color:#6688FF;font-weight:bold">httpGet</span> action will issue an HTTP GET request against a specific port and path in the container.
    - The action **succeeds** if the **HTTP response code is between 200 and 400** (e.g. **200**).
    - The action **fails** if the **HTTP response code is not between 200 and 400** (e.g. **500**).

<img src=../notebook_images/pod_with_probe_actions.png width="800" height="500" style="margin: 1em" />

- Also notice the following settings each probe can define:
  - **initialDelaySeconds** is the initial delay in seconds between the container starting and the probing commencing.
  - **periodSeconds** is the time in seconds between each successive probe.
  - **failureThreshold** is the number of times the probe can be unsuccessful before the probe is considered failed.
    - E.g. `initialDelaySeconds: 5`. `periodSeconds: 2`, `failureThreshold: 3` means:
      - The probing by the `kubelet` commences `5` seconds after the container starts.
      - The `kubelet` will run the probe every `2` seconds after the initial delay.
      - If the probe is unsuccessful `3` times, the probe is considered failed.
      - The first time the probe is run is `initialDeplySeconds + periodSeconds`, i.e. `5 + 2 = 7` seconds.
  - Note that:
    - The first time a probe is run is `initialDeplySeconds + periodSeconds`.
    - The **readinessProbe** and the **livenessProbe** won't run until the **startupProbe** succeeds.
    - A probe is considered **failed** after **failureThreashold** number of **unsuccessful attempts**.
    - A failed **startupProbe** or **livenessProbe** will **restart** the Pod.
    - A failed **readinessProbe** will **not restart** the Pod, but will **stop traffic** being sent to the Pod.

## Ensure a Kubenetes cluster is running

- Use any Kubernetes cluster.
  - In this example, a Minikube cluster with 3 nodes is used: `minikube start --nodes 3`.

**Note! If you are running an Azure Kubernetes Service (AKS), you will also need an Azure Container Registry.**

## Build `probe-example` Docker Image

- Let's build a Docker image of the sample microservice in the folder `probe-example`.
  - The code is in the file `probe-example/src/index.js`.

- The microservice is a Node.js Express Web Server that does the following:
  1. Set timestamp `startupTimestamp` of when the microservice started.
  2. Wait (sleep) for `10` seconds to simulate startup time (initializing the microservice).
  3. Write file `/tmp/started` to indicate the microservice has started.
     - This is what the `startupProbe` checks.
  4. Wait (sleep) for `10` seconds to simulate a delay before the microservice is ready to accept traffic.
  5. Start the Express Web Server listening on port `3000` to indicate the microservie is ready to accept traffic
     - This is what the `readinessProbe` checks.
  6. Respond to HTTP GET requests on route (path) `/healthz` to indicate a state of liveness
     - This is what the `livenessProbe` checks.
     - If within `40` seconds of the microservice's `startupTimestamp`, return status `200` (ok).
     - If **not** within `40` seconds of the microservice's `startupTimestamp`, return status `500` (internal server error).
- The file `probe-example/src/index.js` also outputs log information using `console.log()` and `console.error()`.

**Note! If you are running an Azure Kubernetes Service (AKS), you will need to change the Container Registy's URL below**

- **E.g. use `-t $CONTAINER_REGISTRY_LOGIN_SERVER/probe-example` but with your own Container Registry's details**

In [1]:
!docker build -t probe-example -f probe-example/Dockerfile probe-example/.

failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon  50.69kB
Step 1/6 : FROM node:19.9.0-alpine
 ---> e2a8cc97f817
Step 2/6 : WORKDIR /usr/src/app
 ---> Using cache
 ---> 4cb91cb53a6b
Step 3/6 : COPY package*.json ./
 ---> Using cache
 ---> d9668cc525c6
Step 4/6 : RUN npm ci --omit=dev
 ---> Using cache
 ---> 2b7a3518da79
Step 5/6 : COPY ./src ./src
 ---> 9c81cc664e77
Step 6/6 : CMD npm start
 ---> Running in 686f12714795
Removing intermediate container 686f12714795
 ---> 4238337156c2
Successfully built 4238337156c2
Successfully tagged probe-example:latest


## Load the Docker image from the host machine into the Minikube nodes

- The command below will load the image from your host machine into the Minikube nodes.

**Note! If you are using an Azure Kubernetes Service (AKS), you need to push the image to the Container Registry**

- **E.g., issue the equivalent commands below (but use your own Container Registry's details)**
  ```bash
  docker login $CONTAINER_REGISTRY_LOGIN_SERVER -u $CONTAINER_REGISTRY_USERNAME -p $CONTAINER_REGISTRY_PASSWORD
  docker push $CONTAINER_REGISTRY_LOGIN_SERVER/probe-example
  ```

**Note! If you are using another Kubernetes cluster (besides Minikube or AKS), you need to make the image avaialble there**

In [2]:
!minikube image load probe-example:latest
!minikube image ls | grep probe-example

docker.io/library/probe-example:latest


## Remove `probe-example` Docker Image on Host Machine

- The command below will remove the Docker image on the host machine.

**Note! If you are using an Azure Kubernetes Service (AKS), you need to use the Container Registry's tag**

- **E.g., issue the equivalent command below (but use your own Container Registry's details)**
  ```bash
  docker rmi $CONTAINER_REGISTRY_LOGIN_SERVER/probe-example
  ```

**Note! If you are using another Kubernetes cluster (besides Minikube or AKS), you need to use that Container Registry's tag**

In [3]:
!docker rmi probe-example

Untagged: probe-example:latest
Deleted: sha256:4238337156c2a30a0a77cfa47b428a4da5df476cbd9080a921000381dda87d40
Deleted: sha256:9c81cc664e77e6c45e5d471a709f14b55920bfe1d61b350b6b49b699401d21bb
Deleted: sha256:e9c640899a8aa2a858aa85ba79f59852edf0b0d47b8fe7ce63d9e40c79e6acc1


## Watch Events produced by the `probe-example` Pod

- Run this command in a terminal (won't work from a notebook cell)

```bash

# Monitor (watch) events produced by the "probe-example" Pod
kubectl get events --watch --field-selector involvedObject.name=probe-example
```

## Deploy the Pod `probe-example`

- The definition of the Pod is in the file `manifests/probe-example.yaml`.
- The Pod is named `probe-example`.
- It's container uses the Docker image `probe-example` (that we built above).
  
  - It listens on port `3000`.
  - It defines three probes `startupProbe`, `readinessProbe` and `livenessProbe` as below:

    ```bash
    startupProbe:              # The startupProbe checks for (lists) the file /tmp/started
      exec:
        command:
        - cat
        - /tmp/started
      initialDelaySeconds: 5   # The initial startupProbe is run 5 + 4 = 9 seconds after the Pod starts
      periodSeconds: 4         # After the initial startupProbe, the startupProbe is run every 4 seconds
      failureThreshold: 10     # If the startupProbe in unsuccessful 10 times => startupProbe failed (Pod restarted)
    
    readinessProbe:            # The readinessProbe checks if TCP port 3000 is open
      tcpSocket:
        port: 3000
      initialDelaySeconds: 5   # The initial readinessProbe is run 5 + 2 = 7 seconds after the startupProbe succeeds
      periodSeconds: 2         # After the initial readinessProbe, the readinessProbe is run every 2 seconds
      failureThreshold: 10     # If the readinessProbe in unsuccessful 10 times => readinessProbe failed (traffic stopped)
    
    livenessProbe:             # The livenessProbe issues HTTP GET request to path /healthz on port 3000
      httpGet:
        path: /healthz
        port: 3000
      initialDelaySeconds: 25  # The initial livenessProbe is run 25 + 2 = 7 seconds after the startupProbe succeeds
      periodSeconds: 2         # After the initial livenessProbe, the livenessProbe is run every 2 seconds
      failureThreshold: 6      # If the livenessProbe in unsuccessful 6 times => livenessProbe has failed (Pod restarted)
      ```

In [4]:
!kubectl apply -f manifests/probe-example.yaml

pod/probe-example created


## Watch Information produced by the `probe-example` Pod

- Run this command in a terminal (won't work from a notebook cell)

```bash

# Monitor (watch) logs (stdout, stderr) produced by the "probe-example" Pod
kubectl get pod probe-example -o wide  --watch
```

## Watch Logs produced by the `probe-example` Pod

- Run this command in a terminal (won't work from a notebook cell)
- The command `kubectl logs probe-example --follow` will watch (follow) the log continuously.
- The command `kubectl logs probe-example --follow --tail 10` will only show the latest `10` log lines.

```bash

# Monitor (watch) logs (stdout, stderr) produced by the "probe-example" Pod
kubectl logs probe-example --follow
```

## Check the Events produced by `probe-example` Pod in the terminal

- You should see something similar to the output below
  - First the `probe-example` image is pulled from its Container Registry (if necessary).
  - Then the `probe-example` container is created and started.
  - The `kubelet` starts runing the `startupProbe` after `initialDelaySeconds + periodSeconds`, and then every `periodSeconds`.
  - Initially, the file `/tmp/started` doesn't exist in the container, so the `startupProbe` is unsuccessful.
    - The `kubelet` will retry the `startupProbe` at most `failureThreshold` times (then restart the Pod).
  - Eventually, the `startupProbe` succeeds (the number of attempts never reaches `failureThreshold`).
    - The `kubelet` starts running the `readinessProbe` and `livenessProbe`.
      - The `kubelet` starts runing the `readinessProbe` after `initialDelaySeconds + periodSeconds`, and then every `periodSeconds`.
      - The `kubelet` starts runing the `livenessProbe` after `initialDelaySeconds + periodSeconds`, and then every `periodSeconds`.
  - Initially, TCP port `3000` isn't open in the container, so the `readinessProbe` is unsuccessful.
    - The `kubelet` will retry the `readinessProbe` at most `failureThreshold` times (then stop sending traffic to the Pod).
  - Eventually, the `readinessProbe` succeeds (the number of attempts never reaches `failureThreshold`).
  - Initially, the HTTP GET request to route (path) `/healthz` on port `3000` for the `livenessProbe` is successful (no event registered).
  - Eventually, the `livenessProbe` is unsuccessfull.
    - The `kubelet` will retry the `livenessProbe` at most `failureThreshold` times (then restart the Pod).
    - Since the `failureThreshold` is reached, the `kubelet` restarts the Pod.
  - This pattern then keeps repeating.

```bash
LAST SEEN   TYPE      REASON      OBJECT              MESSAGE
0s          Normal    Scheduled   pod/probe-example   Successfully assigned default/probe-example to minikube-m02

0s          Normal    Pulled      pod/probe-example   Container image "probe-example:latest" already present on machine
0s          Normal    Created     pod/probe-example   Created container probe-example
0s          Normal    Started     pod/probe-example   Started container probe-example
0s          Warning   Unhealthy   pod/probe-example   Startup probe failed: cat: can't open '/tmp/started'
0s          Warning   Unhealthy   pod/probe-example   Startup probe failed: cat: can't open '/tmp/started'
0s          Warning   Unhealthy   pod/probe-example   Readiness probe failed: dial tcp 10.244.1.8:3000: connection refused
0s          Warning   Unhealthy   pod/probe-example   Readiness probe failed: dial tcp 10.244.1.8:3000: connection refused
0s          Warning   Unhealthy   pod/probe-example   Readiness probe failed: dial tcp 10.244.1.8:3000: connection refused
0s          Warning   Unhealthy   pod/probe-example   Readiness probe failed: dial tcp 10.244.1.8:3000: connection refused
0s          Warning   Unhealthy   pod/probe-example   Liveness probe failed: HTTP probe failed with statuscode: 500
0s          Warning   Unhealthy   pod/probe-example   Liveness probe failed: HTTP probe failed with statuscode: 500
0s          Warning   Unhealthy   pod/probe-example   Liveness probe failed: HTTP probe failed with statuscode: 500
0s          Warning   Unhealthy   pod/probe-example   Liveness probe failed: HTTP probe failed with statuscode: 500
0s          Warning   Unhealthy   pod/probe-example   Liveness probe failed: HTTP probe failed with statuscode: 500
0s          Warning   Unhealthy   pod/probe-example   Liveness probe failed: HTTP probe failed with statuscode: 500
0s          Normal    Killing     pod/probe-example   Container probe-example failed liveness probe, will be restarted

0s          Normal    Pulled      pod/probe-example   Container image "probe-example:latest" already present on machine
0s          Normal    Created     pod/probe-example   Created container probe-example
0s          Normal    Started     pod/probe-example   Started container probe-example
0s          Warning   Unhealthy   pod/probe-example   Startup probe failed: cat: can't open '/tmp/started'
0s          Warning   Unhealthy   pod/probe-example   Startup probe failed: cat: can't open '/tmp/started'
0s          Warning   Unhealthy   pod/probe-example   Readiness probe failed: dial tcp 10.244.1.8:3000: connection refused
0s          Warning   Unhealthy   pod/probe-example   Readiness probe failed: dial tcp 10.244.1.8:3000: connection refused
0s          Warning   Unhealthy   pod/probe-example   Readiness probe failed: dial tcp 10.244.1.8:3000: connection refused
0s          Warning   Unhealthy   pod/probe-example   Readiness probe failed: dial tcp 10.244.1.8:3000: connection refused
```

## Describe the Pod (detailed Pod information)

- The description of a Pod contains similar `event` information as above at the bottom of its listing.

In [2]:
!kubectl describe pod probe-example

Name:             probe-example
Namespace:        default
Priority:         0
Service Account:  default
Node:             minikube-m02/192.168.49.3
Start Time:       Wed, 14 Feb 2024 13:49:30 +0100
Labels:           app=probe-example
Annotations:      <none>
Status:           Running
IP:               10.244.1.8
IPs:
  IP:  10.244.1.8
Containers:
  probe-example:
    Container ID:   docker://a7d623951e6be0a4dd822d8b4534e757bc697381ebe5e7c9fbd60412b2a49c99
    Image:          probe-example:latest
    Image ID:       docker://sha256:4238337156c2a30a0a77cfa47b428a4da5df476cbd9080a921000381dda87d40
    Port:           3000/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Wed, 14 Feb 2024 13:50:25 +0100
    Last State:     Terminated
      Reason:       Error
      Exit Code:    1
      Started:      Wed, 14 Feb 2024 13:49:30 +0100
      Finished:     Wed, 14 Feb 2024 13:50:25 +0100
    Ready:          False
    Restart Count:  1
    Limits:
      cpu:     250m


## Check the Information produced by `probe-example` Pod in the terminal

- You should see something similar to the output below
  - Initially then the `probe-example` Pod is started, `0` of `1` container (`0/1`) is `READY`.
    - This is because the `probes` aren't successfull until roughly `24` seconds after the Pod has started.
  - Eventually, `1` of `1` container (`1/1`) is `READY`.
    - This is because the `probes` are successfull at roughly `24` seconds after the Pod has started.
  - Lastly, `0` of `1` container (`0/1`) is once again `READY`.
    - SInce the `livenessProbe` has failed `failureThreshold` number of times at roughly `55` seconds after the Pod started.
  - The `kubelet` restarts the Pod.
  - This pattern then keeps repeating.
  - When the Pod has been restarted `5` times (`RESTARTS`), its `STATUS` becomes `CrashLoopBackOff`.
    - The `kubelet` waits roughtly `1` minute before restarting the Pod.
    - The second `CrashLoopBackOff` waits roughly `2` minutes before restarting the Pod.
    - Each successive `CrashLoopBackOff` waits roughly `4` minutes before restarting the Pod.

```bash
NAME            READY   STATUS             RESTARTS         AGE     IP           NODE
probe-example   0/1     Running            0                1s      10.244.1.8   minikube-m02
probe-example   0/1     Running            0                16s     10.244.1.8   minikube-m02
probe-example   1/1     Running            0                24s     10.244.1.8   minikube-m02
probe-example   0/1     Running            1 (0s ago)       55s     10.244.1.8   minikube-m02
probe-example   0/1     Running            1 (13s ago)      68s     10.244.1.8   minikube-m02
probe-example   1/1     Running            1 (23s ago)      78s     10.244.1.8   minikube-m02
probe-example   0/1     Running            2 (0s ago)       109s    10.244.1.8   minikube-m02
probe-example   0/1     Running            2 (15s ago)      2m4s    10.244.1.8   minikube-m02
probe-example   1/1     Running            2 (23s ago)      2m12s   10.244.1.8   minikube-m02
probe-example   0/1     Running            3 (1s ago)       2m44s   10.244.1.8   minikube-m02
probe-example   0/1     Running            3 (13s ago)      2m56s   10.244.1.8   minikube-m02
probe-example   1/1     Running            3 (23s ago)      3m6s    10.244.1.8   minikube-m02
probe-example   0/1     Running            4 (0s ago)       3m37s   10.244.1.8   minikube-m02
probe-example   0/1     Running            4 (15s ago)      3m52s   10.244.1.8   minikube-m02
probe-example   1/1     Running            4 (23s ago)      4m      10.244.1.8   minikube-m02
probe-example   0/1     Running            5 (0s ago)       4m31s   10.244.1.8   minikube-m02
probe-example   0/1     Running            5 (13s ago)      4m44s   10.244.1.8   minikube-m02
probe-example   1/1     Running            5 (23s ago)      4m54s   10.244.1.8   minikube-m02
probe-example   0/1     CrashLoopBackOff   5 (0s ago)       5m25s   10.244.1.8   minikube-m02
probe-example   0/1     Running            6 (85s ago)      6m50s   10.244.1.8   minikube-m02
probe-example   0/1     Running            6 (99s ago)      7m4s    10.244.1.8   minikube-m02
probe-example   1/1     Running            6 (109s ago)     7m14s   10.244.1.8   minikube-m02
probe-example   0/1     CrashLoopBackOff   6 (0s ago)       7m45s   10.244.1.8   minikube-m02
probe-example   0/1     Running            7 (2m49s ago)    10m     10.244.1.8   minikube-m02
probe-example   0/1     Running            7 (3m3s ago)     10m     10.244.1.8   minikube-m02
probe-example   1/1     Running            7 (3m13s ago)    10m     10.244.1.8   minikube-m02
probe-example   0/1     CrashLoopBackOff   7 (0s ago)       11m     10.244.1.8   minikube-m02
probe-example   0/1     Running            8 (5m13s ago)    16m     10.244.1.8   minikube-m02
probe-example   0/1     Running            8 (5m27s ago)    16m     10.244.1.8   minikube-m02
probe-example   1/1     Running            8 (5m37s ago)    17m     10.244.1.8   minikube-m02
probe-example   0/1     Running            9 (0s ago)       17m     10.244.1.8   minikube-m02
probe-example   0/1     Running            9 (15s ago)      17m     10.244.1.8   minikube-m02
probe-example   1/1     Running            9 (23s ago)      18m     10.244.1.8   minikube-m02
probe-example   0/1     CrashLoopBackOff   9 (1s ago)       18m     10.244.1.8   minikube-m02
probe-example   0/1     Running            10 (5m4s ago)    23m     10.244.1.8   minikube-m02
probe-example   0/1     Running            10 (5m21s ago)   23m     10.244.1.8   minikube-m02
probe-example   1/1     Running            10 (5m29s ago)   24m     10.244.1.8   minikube-m02
probe-example   0/1     Running            11 (0s ago)      24m     10.244.1.8   minikube-m02
probe-example   0/1     Running            11 (13s ago)     24m     10.244.1.8   minikube-m02
probe-example   1/1     Running            11 (23s ago)     24m     10.244.1.8   minikube-m02
probe-example   0/1     CrashLoopBackOff   11 (0s ago)      25m     10.244.1.8   minikube-m02```

## Check the Logs produced by `probe-example` Pod in the terminal

- Logs are generated from output to `stdout` (`console.log()`) and `stderr` (`console.error()`) in the microservice's code.
- You should see something similar to the output below.
  - The Microservice is started.
  - An Express Web Service instance is created.
  - The constant `startupTimestamp` is set to the current time.
  - The microservice sleeps for `10000` milliseconds.
  - The file `/tmp/started` is written.
  - The microservice sleeps for `10000` milliseconds.
  - The Express Web Server starts listening on port `3000`.
  - The HTTP GET route `/healthz` is called.
    - With a 200 response up until `40` seconds have elaspsed since startup (`startupTimestamp`).
    - With a 500 response thereafter.
  - The final `npm` messages are received when the `kubelet` restarts (shuts down) the Pod.

```bash
> probe-example@1.0.0 start
> node ./src/index.js

Microservice started.
Created express web server.
Set startupTimestamp to 12:49:34
Sleeping for 10000 milliseconds
Wrote file /tmp/started.
Sleeping for 10000 milliseconds
Started listening on port 3000.
Route /healthz hit at time 12:49:56, Seconds since startup: 22.53
Route /healthz returning status 200
Route /healthz hit at time 12:49:58, Seconds since startup: 24.53
Route /healthz returning status 200
Route /healthz hit at time 12:50:00, Seconds since startup: 26.53
Route /healthz returning status 200
Route /healthz hit at time 12:50:02, Seconds since startup: 28.53
Route /healthz returning status 200
Route /healthz hit at time 12:50:04, Seconds since startup: 30.53
Route /healthz returning status 200
Route /healthz hit at time 12:50:06, Seconds since startup: 32.53
Route /healthz returning status 200
Route /healthz hit at time 12:50:08, Seconds since startup: 34.53
Route /healthz returning status 200
Route /healthz hit at time 12:50:10, Seconds since startup: 36.53
Route /healthz returning status 200
Route /healthz hit at time 12:50:12, Seconds since startup: 38.53
Route /healthz returning status 200
Route /healthz hit at time 12:50:14, Seconds since startup: 40.53
Route /healthz returning status 500
Route /healthz hit at time 12:50:16, Seconds since startup: 42.53
Route /healthz returning status 500
Route /healthz hit at time 12:50:18, Seconds since startup: 44.53
Route /healthz returning status 500
Route /healthz hit at time 12:50:20, Seconds since startup: 46.53
Route /healthz returning status 500
Route /healthz hit at time 12:50:22, Seconds since startup: 48.53
Route /healthz returning status 500
Route /healthz hit at time 12:50:24, Seconds since startup: 50.53
Route /healthz returning status 500
npm ERR! path /usr/src/app
npm ERR! command failed
npm ERR! signal SIGTERM
npm ERR! command sh -c node ./src/index.js

npm ERR! A complete log of this run can be found in: /root/.npm/_logs/2024-02-14T12_49_33_205Z-debug-0.log
```

## Check the Events, Information and Logs for Pod `probe-example` in K9s

- Open K9s in a new Terminal by typing `k9s` in the terminal.
- Type `:events /probe-example` to open the Events view for the `probe-example` Pod.
  - Press `shift + l` to sort the events in descreasing order (`LastSeen`).
- Type `:pod /probe-example` to open the Pods view for the `probe-example` Pod.
  - The latest information about the Pod will be shown on one line.
  - Notice the `STATUS` for the `probe-example` Pod is (eventuall) `CrashLoopBackoff`.
- With the `probe-example` Pod selected in the Pods view, press `l` (for `logs`).
  - The logs for the `probe-example` Pod are displayed.
- Type `:popeye` to open "Popeye" with a status overview of cluster resources.
  - Notice the `probe-example` Pod is failing (`ERROR`).
- Press `ctrl + c` to close K9s.

## Delete the Pod

In [3]:
!kubectl delete -f manifests/probe-example.yaml --force --grace-period=0

pod "probe-example" force deleted


## Stop watching for events produced by the `probe-example` Pod

- Press `ctrl + c` in the terminal watching for events.

## Stop watching for Information produced by the `probe-example` Pod

- If the Pod has been restarted, the terminal will have stopped watching for logs (since the last instance exited).
- Otherwise, press `ctrl + c` in the terminal watching for information.

## Stop watching for logs produced by the `probe-example` Pod

- Press `ctrl + c` in the terminal watching for logs.